In [107]:
# Studio delle relazioni tra generi cinematografici, premi Oscar, attori e valutazioni
#ipotesi: c'è correlazione per un determinato genere in un periodo di tempo a ottenere rating piu alto? o vincere di piu? ovvero, ci sono state delle mode che hanno influenzato i risultati "tecnici"?

import pandas as pd
import plotly.express as px
import psycopg2


def run_query(query):
    try:
        conn = psycopg2.connect(
            dbname="SQLDB",
            user="postgres",
            password="root",
            host="localhost",
            port="5432"
        )
        df = pd.read_sql(query, conn)
        conn.close()
        return df
    except Exception as e:
        print(f"Database error: {e}")
        return pd.DataFrame()
    


# 1: Rating medio per decennio per genere

rating_by_decade_query = """
SELECT 
    g.genre,
    FLOOR(m.year / 10) * 10 AS decade,
    AVG(m.rating) AS avg_rating,
    COUNT(m.id) AS film_count
FROM movies m
JOIN genres g ON m.id = g.id_movie
WHERE m.year IS NOT NULL AND m.rating IS NOT NULL AND m.year > 1910  --1910 per analizzare meglio l'obbiettivo
GROUP BY g.genre, FLOOR(m.year / 10) * 10
"""
rating_by_decade = run_query(rating_by_decade_query)

fig = px.line(
    rating_by_decade,
    x='decade',
    y='avg_rating',
    color='genre',
    template='plotly_dark',
    markers=True,  #pallini per lhover delle label
    labels={'decade': 'Decennio', 'avg_rating': 'Rating Medio'},
    height=1000,  
    width=1200   
)

fig.update_layout(
    title=dict(
        text='Rating medio per decennio per genere',
        y=0.97,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    xaxis=dict(tick0=1900, dtick=10),
    yaxis=dict(range=[2.8, rating_by_decade['avg_rating'].max()+0.04]), 
    legend=dict(
        title='Generi',
        yanchor='top',
        xanchor='right',
        y=1,
        x=1.1
    )
        
)

fig.show()

C:\Users\aronn\AppData\Local\Temp\ipykernel_18064\2753813331.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [108]:
# 2 Oscar vinti per decennio per genere

#crea una cte con solo i film che hanno vinto oscar, per ogni decennio e genere calcola il numero di film e fa join con la prima lista su name. serve la join perche oscar non ha generi 

oscars_by_decade_query = """
WITH oscar_films AS (
    SELECT DISTINCT film as movie_title
    FROM oscar_awards
    WHERE winner = true)
SELECT 
    g.genre,
    FLOOR(m.year / 10) * 10 AS decade,
    COUNT(DISTINCT CASE WHEN o.movie_title IS NOT NULL THEN m.id END) AS genre_oscars,
    COUNT(DISTINCT m.id) AS total_films
FROM movies m
JOIN genres g ON m.id = g.id_movie
LEFT JOIN oscar_films o ON LOWER(m.name) = LOWER(o.movie_title)  --lower anche se non è necessario usato in tutti  
WHERE m.year IS NOT NULL AND m.year > 1900               
GROUP BY g.genre, FLOOR(m.year / 10) * 10
HAVING COUNT(DISTINCT m.id) > 5
"""
oscars_by_decade = run_query(oscars_by_decade_query)

fig = px.line(
    oscars_by_decade,
    x='decade',
    y='genre_oscars',
    color='genre',
    labels={'decade': 'Decennio', 'genre_oscars': 'Film vincitori di Oscar'},
    template='plotly_dark',
    markers=True,
    height=1000,  
    width=1200   
)

fig.update_layout(
       title=dict(
        text='Numero di Film Vincitori di Oscar per Genere nei Decenni',
        y=0.95,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    yaxis=dict(range=[0, oscars_by_decade['genre_oscars'].max()-175]), 
    xaxis=dict(range=[1909, oscars_by_decade['decade'].max()+10]),
    legend=dict(
        title='Generi',
        yanchor='top',
        y=0.7,
        xanchor='right',
        x=1.1
    )    
)

fig.add_annotation(   #aggiunta annotazione per tagliare grafico per un punto solo di un gener/linea sola
    x=2010, 
    y=243,
    text="Picco negli anni per i generi Drammatici, 425 nel 2010",
    showarrow=False,
    bordercolor='#ffd700',
    borderpad=4,
    opacity=0.9
)

fig.show()

C:\Users\aronn\AppData\Local\Temp\ipykernel_18064\2753813331.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [109]:
# 3 Generi con piu oscar vinti in totale a barre

genre_oscar_query = """
WITH oscar_films AS (
    SELECT DISTINCT film as movie_title
    FROM oscar_awards
    WHERE winner = true)
SELECT
    g.genre,
    COUNT(DISTINCT CASE WHEN o.movie_title IS NOT NULL THEN m.id END) as oscar_films,
    COUNT(DISTINCT m.id) as total_films,
    AVG(m.rating) as avg_rating  --usato in 4 non in questa cella
FROM movies m
JOIN genres g ON m.id = g.id_movie
LEFT JOIN oscar_films o ON LOWER(m.name) = LOWER(o.movie_title)   --lower dato che movies  e oscar_films sono da fonti di dati diverse
GROUP BY g.genre
ORDER BY oscar_films DESC
"""

genre_oscar = run_query(genre_oscar_query)

fig = px.bar(
    genre_oscar,
    x='oscar_films',
    y='genre',
    orientation='h',
    title='Generi con più Oscar vinti',
    labels={
        'oscar_films': 'Numero di Oscar vinti',
        'genre': 'Genere'
    },
    color_discrete_sequence=['royalblue'],   #colore uniforme
    height=800,
    width=800,
    template='plotly_dark'
)

fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        dtick=100,
        title='Numero di Oscar vinti'
    ),
    yaxis={'title': 'Genere'},
    coloraxis_showscale=False  #nasconde la legenda scala colori
)

fig.show()

C:\Users\aronn\AppData\Local\Temp\ipykernel_18064\2753813331.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [110]:
## 4 relazione tra rating medio e probabilità di vincere un Oscar per genere

genre_oscar['oscar_ratio'] = genre_oscar['oscar_films'] / genre_oscar['total_films'] #genre oscar da n.3, niente round perche il ratio dipende troppo da piccoli decimali

fig = px.scatter(
    genre_oscar,
    x='avg_rating',
    y='oscar_ratio',
    size='total_films',
    color='genre',
    hover_name='genre',
    labels={
        'avg_rating': 'Rating Medio',
        'oscar_ratio': 'Probabilità di Vincere Oscar',
        'total_films': 'Numero Film',
        'genre': 'Genere'
    },
    title='Relazione tra Rating Medio e Probabilità di Vincere Oscar',
    size_max=40 
)

fig.update_traces(
    marker=dict(
        sizemin=5,
        sizeref=2.*max(genre_oscar['total_films'])/(100.**2)+100
    ),
)

fig.update_layout(
    height=700,
    width=900,
    template='plotly_dark',
    legend_title_text='Generi',
    xaxis={'title':'Rating Medio'},
    yaxis={'title':'Probabilità di Vincere Oscar'},
    title=dict(
        y=0.95,
        x=0.5,
        xanchor='center'
    )
)

fig.show()

In [111]:
""" dai 4 grafici visti parrebbe sconnessa la relazione tra genere e periodo che ha influenzato i risultati in quanto: nonostante il genere drama sia quello che ha vinto piu oscar di tutti dagli anni '50 non è il genere 
##con il rating migliore durante lasso temporale analizzato, il rating migliore nel tempo lha tenuto il genere documentary che pero non rientra nemmeno nelal top 3 di top generi vincitori di oscars.
#d'altro canto il genere che sembra aver avuto una piu correlations tra rating alto e percentuale di vincita oscar è il genere war, forse dato dai pochi film paragonandolo agli altri generi, Drama difatti nonostante il numero maggiore
#di film è quasi al centro dell'ultimo grafico mostrando quindi una correlazione tra rating e prob. vincita.
Magari è derivato dagli attori che in alcuni generi eseguono performance magistrali permettendo ad un film di vincere o ottenere rating alti ?"""

" dai 4 grafici visti parrebbe sconnessa la relazione tra genere e periodo che ha influenzato i risultati in quanto: nonostante il genere drama sia quello che ha vinto piu oscar di tutti dagli anni '50 non è il genere \n##con il rating migliore durante lasso temporale analizzato, il rating migliore nel tempo lha tenuto il genere documentary che pero non rientra nemmeno nelal top 3 di top generi vincitori di oscars.\n#d'altro canto il genere che sembra aver avuto una piu correlations tra rating alto e percentuale di vincita oscar è il genere war, forse dato dai pochi film paragonandolo agli altri generi, Drama difatti nonostante il numero maggiore\n#di film è quasi al centro dell'ultimo grafico mostrando quindi una correlazione tra rating e prob. vincita.\nMagari è derivato dagli attori che in alcuni generi eseguono performance magistrali permettendo ad un film di vincere o ottenere rating alti ?"

In [116]:

## 5 Oscar vinti da attori per genere (top 20)

#seleziona tutti gli attori che hanno avuto un ruolo primario per l'oscar, join con movie su name per arrivare al genere e dividere 
actor_genre_perf_query = """  SELECT
    s.actor_name AS actor,
    g.genre,
    s.movies_count AS film_count,
    COUNT(DISTINCT CASE WHEN o.winner = true AND o.name = s.actor_name 
                        AND o.category IN (
                            'ACTOR', 'ACTRESS',
                            'ACTOR IN A LEADING ROLE', 'ACTRESS IN A LEADING ROLE',      --selezione di tipi di oscar per attori
                            'ACTOR IN A SUPPORTING ROLE', 'ACTRESS IN A SUPPORTING ROLE')
                   THEN o.id END) AS actor_oscars
FROM actor_summaries s
JOIN actors a ON s.actor_name = a.name
JOIN movies m ON a.id_movie = m.id
JOIN genres g ON m.id = g.id_movie    --necessaria la join con generi perche actor summaries non ha il genere preciso del film con cui ha vinto l'oscar
LEFT JOIN oscar_awards o ON LOWER(m.name) = LOWER(o.film) where o.winner = true     --ripetuto controllo per velocizzare evitando inutili join
GROUP BY s.actor_name, g.genre, s.movies_count
ORDER BY actor_oscars  DESC 
LIMIT 27
""" #limit 27 per come funzionano i group se un attore ha oscar per generi diversi lo conta come separati
winners = run_query(actor_genre_perf_query)

fig = px.bar(
    winners,
    x='actor',
    y='actor_oscars',
    color='genre',
    title='Relazione tra Attori e Oscar Vinti per Genere',
    hover_data=['film_count'],
    labels={
        'actor': 'Attore',
        'actor_oscars': 'Oscar Vinti',
        'genre': 'Genere',
        'film_count': 'Numero Film'
    },
)

fig.update_layout(
    height=600,
    width=1000,  
    xaxis={'tickangle': 45}, 
    yaxis={'dtick': 1}, 
    template='plotly_dark',
   legend=dict(
        title='Genere',
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='right',
        x=1
    )
)



C:\Users\aronn\AppData\Local\Temp\ipykernel_18064\2753813331.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [115]:
#6 Correlazione tra migliori attori e rating medio dei film

raw_query = """
SELECT 
    a.name AS actor,
    a.role AS role,
    m.id AS movie_id,
    m.name AS movie_name,
    m.rating,
    m.minute,
    actor_summaries.average_rating AS overall_avg_rating,
    actor_summaries.movies_count AS total_films,
    actor_summaries.genres AS full_genres,
    g.genre
FROM actors a
JOIN actor_summaries 
    ON actor_summaries.actor_name = a.name 
    AND actor_summaries.movies_count >= 15        --rimozioni attori che hanno lavorato poco
    AND actor_summaries.average_rating >= 3.5    --filtro per lavorare con meno dati filtrando per una base di rating medio
    AND actor_summaries.movies_count < 200       --rimozione attori che hanno fatto tanti film "finti"
JOIN movies m ON a.id_movie = m.id
JOIN genres g ON m.id = g.id_movie
WHERE m.rating IS NOT NULL AND m.minute >= 60   --filtro per film "veri"
"""
raw_data = run_query(raw_query)

mask = ~raw_data['role'].str.lower().str.contains('voice|uncredited|cameo|archive', na=False)
filtered_data = raw_data[mask]

actor_stats = filtered_data[['actor', 'total_films', 'overall_avg_rating', 'full_genres']].drop_duplicates(subset='actor')
actor_stats['n_genres'] = actor_stats['full_genres'].apply(
    lambda x: len(x) if isinstance(x, (list, set, tuple)) else (
        len(str(x).strip('{}').split(',')) if pd.notnull(x) else 0
    )
)
actor_stats = actor_stats[actor_stats['n_genres'] >= 4]  #filtro per attori che hanno recitato in meno di 4 generi, contando quanti generi dalla stringa divisa a , presente in actor_summaries

agg_data = filtered_data.groupby(['actor', 'genre']).agg( #conteggio iflm per e rating medio per genere per popolare hover del grafico e ordinare per trovare i top actors
    genre_film_count=('movie_id', 'nunique'),
    genre_avg_rating=('rating', 'mean')
).reset_index()

genre_scores = agg_data.groupby('actor').apply(
    lambda df: np.average(df['genre_avg_rating'], weights=df['genre_film_count'])
).reset_index(name='genre_score')

actor_stats = actor_stats.merge(genre_scores, on='actor')
    
actor_stats['weighted_score'] = (           #forumla e esi decisi da chatgpt dopo innumerevoli prove per torvare un buon modo/realistico per trovare i top attori coi dati che abbiamo
    actor_stats['overall_avg_rating'] * 0.5 +
    np.log1p(actor_stats['total_films']) * 0.25 +
    actor_stats['n_genres'] * 0.15 +
    actor_stats['genre_score'] * 0.1
)

top_actors = actor_stats.nlargest(20, 'weighted_score')['actor']

agg_data = agg_data[agg_data['actor'].isin(top_actors)]
agg_data = agg_data.merge(actor_stats, on='actor')

agg_data['genre_avg_rating'] = agg_data['genre_avg_rating'].round(2)
agg_data['overall_avg_rating'] = agg_data['overall_avg_rating'].round(2)

import plotly.express as px

fig = px.scatter(
    agg_data,
    x='genre',
    y='actor',
    size='genre_film_count',
    color='genre_avg_rating',
    color_continuous_scale='Rainbow',
    hover_data={
        'genre_avg_rating': True,
        'genre_film_count': True,
        'overall_avg_rating': True,
        'total_films': True,
        'n_genres': True,
        'genre_score': True,
        'weighted_score': False
    },
    labels={
        'genre': 'Genere',
        'actor': 'Attore',
        'genre_film_count': 'Numero Film nel Genere',
        'genre_avg_rating': 'Rating Medio nel Genere'
    }
)

fig.update_layout(
    height=800,
    width=1100,
    template='plotly_dark',
    xaxis={'categoryorder': 'total descending', 'tickangle': 45},
    yaxis={'categoryorder': 'total ascending'},
    title=dict(
        text='Top 20 Attori (Priorità a Qualità e Quantità Totale)',
        y=0.95,
        x=0.5,
        xanchor='center',
        yanchor='top'
    )
)

fig.show()


C:\Users\aronn\AppData\Local\Temp\ipykernel_18064\2753813331.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\aronn\AppData\Local\Temp\ipykernel_18064\365771643.py:43: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [114]:
"""Come si vede in questo ultimo grafico non compaiono gli attori gli attori vincitori di oscar scollegando ancora di piu rating dagli oscar, si potrebbe dedurre in quanto il rating "affibiato" agli attori 
non è "il loro" ma dei film dove hanno recitato e per il calcolo ci sono altre cose da tenere conto oltre alla loro recitazione. Si puo vedere pero che, perlomeno secondo i criteri scelti per ottenerli da questi dati,
 i "migliori" attori solo collegati a rating maggiori per i generi piu "tipici" di film come frama e action, dove difatti hanno tutti piu recitazioni. """

'Come si vede in questo ultimo grafico non compaiono gli attori gli attori vincitori di oscar scollegando ancora di piu rating dagli oscar, si potrebbe dedurre in quanto il rating "affibiato" agli attori \nnon è "il loro" ma dei film dove hanno recitato e per il calcolo ci sono altre cose da tenere conto oltre alla loro recitazione. Si puo vedere pero che, perlomeno secondo i criteri scelti per ottenerli da questi dati,\n i "migliori" attori solo collegati a rating maggiori per i generi piu "tipici" di film come frama e action, dove difatti hanno tutti piu recitazioni. '